In [ ]:
import pandas as pd
import sqlite3

In [ ]:
def load_orders(csv_path: str) -> pd.DataFrame:
    """Load orders CSV file"""
    return pd.read_csv(csv_path)

In [ ]:
def load_users(json_path: str) -> pd.DataFrame:
    """Load users JSON file"""
    return pd.read_json(json_path)

In [ ]:
def load_restaurants(sql_path: str) -> pd.DataFrame:
    """
    Load restaurants data from SQL file by executing it
    into an in-memory SQLite database
    """
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()

    with open(sql_path, "r", encoding="utf-8") as file:
        sql_script = file.read()

    cursor.executescript(sql_script)

    restaurants_df = pd.read_sql(
        "SELECT * FROM restaurants",
        conn
    )

    conn.close()
    return restaurants_df

In [ ]:
def merge_datasets(
    orders_df: pd.DataFrame,
    users_df: pd.DataFrame,
    restaurants_df: pd.DataFrame
) -> pd.DataFrame:
    """Perform LEFT JOINs to create final dataset"""

    orders_users = pd.merge(
        orders_df,
        users_df,
        on="user_id",
        how="left"
    )

    final_df = pd.merge(
        orders_users,
        restaurants_df,
        on="restaurant_id",
        how="left"
    )

    return final_df

In [ ]:
def main():
    # File paths
    ORDERS_FILE = "orders.csv"
    USERS_FILE = "users.json"
    RESTAURANTS_FILE = "restaurants.sql"

    # Load data
    orders_df = load_orders(ORDERS_FILE)
    users_df = load_users(USERS_FILE)
    restaurants_df = load_restaurants(RESTAURANTS_FILE)

    # Merge data
    final_df = merge_datasets(
        orders_df,
        users_df,
        restaurants_df
    )

    # Output
    print("Final dataset preview:")
    print(final_df.head())

    print("\nFinal dataset info:")
    print(final_df.info())

    # Save final dataset
    final_df.to_csv("final_orders_dataset.csv", index=False)
    print("\n✅ final_orders_dataset.csv created successfully")

In [ ]:
if __name__ == "__main__":
    main()